In [2]:
# Importar dependencias
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
# Determinando los puntos de las articulaciones
# Reemplaza con la ruta de tu archivo de video
video_path = 'C:/Users/USUARIO/Proyectos_IA/Datasets/videos_gym/press_incorrectos/press_jhon_incorrecto.mp4'  
cap = cv2.VideoCapture(video_path)

# cap = cv2.VideoCapture(0)

# Configurar instancia de mediapipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Cambiar el color de la imagen a RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Hacer detección
        results = pose.process(image)
    
        # Cambiar el color de nuevo a BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extraer puntos de referencia
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        
        # Detecciones de renderizado
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\Users\USUARIO\anaconda3\envs\IA_env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[x: 0.485908031
y: 0.277731538
z: 1.01571
visibility: 0.999999642
, x: 0.499299496
y: 0.275493771
z: 1.02617848
visibility: 0.999999762
, x: 0.511924446
y: 0.274527609
z: 1.0265305
visibility: 0.999999642
, x: 0.536084652
y: 0.26792711
z: 1.02583885
visibility: 0.999999642
, x: 0.469168335
y: 0.277810812
z: 1.04927778
visibility: 0.999999762
, x: 0.462509304
y: 0.27742222
z: 1.04932296
visibility: 0.999999762
, x: 0.45610851
y: 0.276961446
z: 1.04955626
visibility: 0.999999881
, x: 0.573761225
y: 0.271102816
z: 1.00593567
visibility: 0.999999762
, x: 0.447062314
y: 0.290493727
z: 1.10737848
visibility: 0.999999881
, x: 0.511280298
y: 0.290029
z: 1.00271606
visibility: 0.999999404
, x: 0.476694167
y: 0.292771101
z: 1.03013635
visibility: 0.999999762
, x: 0.654751837
y: 0.296813935
z: 0.911443174
visibility: 0.999997616
, x: 0.407790363
y: 0.331213832
z: 0.984033167
visibility: 0.999999642
, x: 0.800951242
y: 0.324115276
z: 1.15838456
visibility: 0.96563518
, x: 0.277724564
y: 0.39733108

In [4]:
len(landmarks)

33

In [5]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

0.9999921917915344

In [6]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility

0.999988853931427

In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.765910327
y: 0.368836552
z: 0.797661424
visibility: 0.96308589

In [8]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.279342711
y: 0.318025023
z: 0.726717055
visibility: 0.985623181

In [9]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.815836668
y: 0.296957016
z: 0.909920931
visibility: 0.988831103

In [10]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

x: 0.246630535
y: 0.239401966
z: 0.750589132
visibility: 0.993125677

In [11]:
# Calcular ángulos para el press de banca
def calculate_press_angle(shoulder, elbow, wrist):
    """
    Calcula el ángulo entre el hombro, codo y muñeca.
    shoulder: Coordenadas del hombro (punto fijo en el cuerpo)
    elbow: Coordenadas del codo (punto medio)
    wrist: Coordenadas de la muñeca (punto final)
    """
    
    # Convertir los puntos a arrays de NumPy
    shoulder = np.array(shoulder)  # Punto de referencia (hombro)
    elbow = np.array(elbow)        # Punto medio (codo)
    wrist = np.array(wrist)        # Punto final (muñeca)
    
    # Calcular el ángulo en radianes
    radians = np.arctan2(wrist[1] - elbow[1], wrist[0] - elbow[0]) - np.arctan2(shoulder[1] - elbow[1], shoulder[0] - elbow[0])
    angle = np.abs(radians * 180.0 / np.pi)  # Convertir a grados
    
    # Asegurarse de que el ángulo esté dentro del rango de 0-180 grados
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

In [12]:
# Obtener coordenadas de los puntos clave del brazo izquierdo
left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [13]:
# Obtener coordenadas de los puntos clave del brazo derecho
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

In [14]:
# Calcular ángulo para brazo izquierdo
calculate_press_angle(left_shoulder, left_elbow, left_wrist)

96.67893854135855

In [15]:
# Calcular ángulo para brazo derecho
calculate_press_angle(right_shoulder, right_elbow, right_wrist)

115.01008111140374

In [16]:
# Multiplicar las coordenadas del codo izquierdo
tuple(np.multiply(left_elbow, [640, 480]).astype(int))

(490, 177)

In [17]:
# Multiplicar las coordenadas del codo derecho
tuple(np.multiply(right_elbow, [640, 480]).astype(int))

(178, 152)

In [18]:
# Contador de repeticiones para press de banca
# Reemplaza con la ruta de tu archivo de video
video_path = 'C:/Users/USUARIO/Proyectos_IA/Datasets/videos_gym/press_correctos/press_mancuernas_inclinado_correcto.mp4'  
cap = cv2.VideoCapture(video_path)

# cap = cv2.VideoCapture(0)

# Variables del contador
counter = 0 
stage = None

# Configurar instancia de MediaPipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Cambiar el color de la imagen a RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Hacer detección
        results = pose.process(image)
    
        # Cambiar el color de nuevo a BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Redimensionar el video a 0x0 píxeles
        image = cv2.resize(image, (570, 1000))
        
        # Extraer puntos de referencia
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Obtener coordenadas del brazo izquierdo
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Obtener coordenadas del brazo derecho
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calcular ángulos
            left_angle = calculate_press_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calculate_press_angle(right_shoulder, right_elbow, right_wrist)
            
            # Visualizar ángulos
            left_elbow_coords = tuple(np.multiply(left_elbow, [640, 480]).astype(int))
            right_elbow_coords = tuple(np.multiply(right_elbow, [640, 480]).astype(int))
            
            cv2.putText(image, str(int(left_angle)), 
                        left_elbow_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, str(int(right_angle)), 
                        right_elbow_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Lógica del contador de repeticiones para press de banca
            if left_angle > 100 and right_angle > 100:  # Ambos brazos extendidos (parte superior)
                stage = "up"
            if left_angle < 50 and right_angle < 50 and stage == 'up':  # Ambos brazos en ángulo de 45 grados (parte inferior)
                stage = "down"
                counter += 1
                print(counter)
                       
        except:
            pass
        
        # Renderizar contador de repeticiones
        # Cuadro de estado de configuración
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Mostrar repeticiones
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Mostrar etapa del movimiento
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Detecciones de renderizado
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
